# WideResNet28-10 on CIFAR-10

* https://github.com/google/uncertainty-baselines/blob/master/baselines/cifar/deterministic.py#L145
* WideResNet28-10 on CIFAR-10 모델 학습 + 체크 포인트 저장

## 라이브러리 import 및 flags 선언

In [ ]:
import os
import time
from absl import app
from absl import flags
from absl import logging
import robustness_metrics as rm
import tensorflow as tf
import tensorflow_datasets as tfds
import uncertainty_baselines as ub
import utils  # local file import
from tensorboard.plugins.hparams import api as hp


flags.DEFINE_float('label_smoothing', 0., 'Label smoothing parameter in [0,1].')
flags.register_validator('label_smoothing',
                         lambda ls: ls >= 0.0 and ls <= 1.0,
                         message='--label_smoothing must be in [0, 1].')

# Data Augmentation flags.
flags.DEFINE_bool('augmix', False,
                  'Whether to perform AugMix [4] on the input data.')
flags.DEFINE_integer('aug_count', 1,
                     'Number of augmentation operations in AugMix to perform '
                     'on the input image. In the simgle model context, it'
                     'should be 1. In the ensembles context, it should be'
                     'ensemble_size if we perform random_augment only; It'
                     'should be (ensemble_size - 1) if we perform augmix.')
flags.DEFINE_float('augmix_prob_coeff', 0.5, 'Augmix probability coefficient.')
flags.DEFINE_integer('augmix_depth', -1,
                     'Augmix depth, -1 meaning sampled depth. This corresponds'
                     'to line 7 in the Algorithm box in [4].')
flags.DEFINE_integer('augmix_width', 3,
                     'Augmix width. This corresponds to the k in line 5 in the'
                     'Algorithm box in [4].')

# Fine-grained specification of the hyperparameters (used when FLAGS.l2 is None)
flags.DEFINE_float('bn_l2', None, 'L2 reg. coefficient for batch-norm layers.')
flags.DEFINE_float('input_conv_l2', None,
                   'L2 reg. coefficient for the input conv layer.')
flags.DEFINE_float('group_1_conv_l2', None,
                   'L2 reg. coefficient for the 1st group of conv layers.')
flags.DEFINE_float('group_2_conv_l2', None,
                   'L2 reg. coefficient for the 2nd group of conv layers.')
flags.DEFINE_float('group_3_conv_l2', None,
                   'L2 reg. coefficient for the 3rd group of conv layers.')
flags.DEFINE_float('dense_kernel_l2', None,
                   'L2 reg. coefficient for the kernel of the dense layer.')
flags.DEFINE_float('dense_bias_l2', None,
                   'L2 reg. coefficient for the bias of the dense layer.')


flags.DEFINE_bool('collect_profile', False,
                  'Whether to trace a profile with tensorboard')

FLAGS = flags.FLAGS

## 하이퍼 파라미터 세팅

In [ ]:
def _extract_hyperparameter_dictionary():
  """Create the dictionary of hyperparameters from FLAGS."""
  flags_as_dict = FLAGS.flag_values_dict()
  hp_keys = ub.models.models.wide_resnet.HP_KEYS
  hps = {k: flags_as_dict[k] for k in hp_keys}
  return hps

## main 함수

### log 값들을 처리하는 로깅 처리기 선언

In [ ]:
def main(argv):
  fmt = '[%(filename)s:%(lineno)s] %(message)s'
  formatter = logging.PythonFormatter(fmt)
  logging.get_absl_handler().setFormatter(formatter)
  del argv  # argv 는 사용 안하므로 제거

### output 디렉토리,  global seed 선언

In [ ]:
  tf.io.gfile.makedirs(FLAGS.output_dir) # 디렉토리 작성(상위 or 중간)
  logging.info('Saving checkpoints at %s', FLAGS.output_dir) # 로그 정보를 알려줌(출력X)
  tf.random.set_seed(FLAGS.seed) # global한 random seed 생성

  data_dir = FLAGS.data_dir # data가 저장되어있는 data_dir 선언

### GPU or TPU 환경 설정

In [ ]:
  if FLAGS.use_gpu: #GPU 환경 설정
    logging.info('Use GPU')
    strategy = tf.distribute.MirroredStrategy()
  else: # TPU 환경 설정
    logging.info('Use TPU at %s',
                 FLAGS.tpu if FLAGS.tpu is not None else 'local')
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu)
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
  ds_info = tfds.builder(FLAGS.dataset).info # tfds.core.DatasetBuilder의 dataset(=cifar10) 가져와서 정보 저장
  batch_size = FLAGS.per_core_batch_size * FLAGS.num_cores # batch_size 설정, 64*8
  train_dataset_size = ( # train_dataset_size 선언, 50000 * 1.0
      ds_info.splits['train'].num_examples * FLAGS.train_proportion)
  steps_per_epoch = int(train_dataset_size / batch_size) # steps_per_epoch 선언, 50000/(64*8)
  logging.info('Steps per epoch %s', steps_per_epoch)# 로그 정보를 알려줌(출력X)
  logging.info('Size of the dataset %s', ds_info.splits['train'].num_examples)# 로그 정보를 알려줌(출력X)
  logging.info('Train proportion %s', FLAGS.train_proportion)# 로그 정보를 알려줌(출력X)
  steps_per_eval = ds_info.splits['test'].num_examples // batch_size #Steps_per_epoch 선언, 10000//(64*8)
  num_classes = ds_info.features['label'].num_classes #num_classes 선언, 10

### aug_params 선언

In [ ]:
  aug_params = {
      'augmix': FLAGS.augmix, #입력 데이터에 augmix를 수행할지 여부, False
      'aug_count': FLAGS.aug_count, #augmix에서 수행할 증강 작업 수, 1
      'augmix_depth': FLAGS.augmix_depth, #augmix 확률계수, 0.5
      'augmix_prob_coeff': FLAGS.augmix_prob_coeff, #표본의 깊이, -1
      'augmix_width': FLAGS.augmix_width, # augmix 폭, 3
  }

In [ ]:
  seeds = tf.random.experimental.stateless_split( #2개의 seed 생성, [1769886085, 86449935]
      [FLAGS.seed, FLAGS.seed + 1], 2)[:, 0]

### 데이터셋 생성

In [ ]:
  train_builder = ub.datasets.get( #train_dataset 생성
      FLAGS.dataset,
      data_dir=data_dir,
      download_data=FLAGS.download_data,
      split=tfds.Split.TRAIN,
      seed=seeds[0],
      aug_params=aug_params,
      validation_percent=1. - FLAGS.train_proportion,)
  train_dataset = train_builder.load(batch_size=batch_size) # 생성된 train 데이터로 train_dataset 선언
  validation_dataset = None # validation_dataset 선언, None, (trainproportion이 1.0이다)
  steps_per_validation = 0 # steps_per_validation 선언, 0
  clean_test_builder = ub.datasets.get( #test_dataset 생성
      FLAGS.dataset,
      split=tfds.Split.TEST,
      data_dir=data_dir)
  clean_test_dataset = clean_test_builder.load(batch_size=batch_size) # 생성된 test 데이터로 clean_test_dataset 선언

### 분산환경에 맞춰 다시 데이터셋 선언

In [ ]:
  train_dataset = strategy.experimental_distribute_dataset(train_dataset)
  test_datasets = {
      'clean': strategy.experimental_distribute_dataset(clean_test_dataset),
  }
  steps_per_epoch = train_builder.num_examples // batch_size #steps_per_epoch 선언
  steps_per_eval = clean_test_builder.num_examples // batch_size #steps_per_eval 선언
  num_classes = 100 if FLAGS.dataset == 'cifar100' else 10 #num_classes 선언, 10

### 지정된 디렉토리에 저장될 요약 파일 작성

In [ ]:
  summary_writer = tf.summary.create_file_writer( #지정된 디렉토리에 저장될 요약 파일 작성
      os.path.join(FLAGS.output_dir, 'summaries'))

### 여러 장치로 분산시켜 훈련(TPU or GPU)

In [ ]:
  with strategy.scope(): #환경설정된 gpu or tpu 오픈
    logging.info('Building ResNet model') # 로그 정보를 알려줌(출력X)
    model = ub.models.wide_resnet( #wide_resnet28-10 모델 구축
        input_shape=(32, 32, 3),
        depth=28,
        width_multiplier=10,
        num_classes=num_classes,
        l2=FLAGS.l2,
        hps=_extract_hyperparameter_dictionary(),
        seed=seeds[1])
    logging.info('Model input shape: %s', model.input_shape) # 로그 정보를 알려줌(출력X)
    logging.info('Model output shape: %s', model.output_shape) # 로그 정보를 알려줌(출력X)
    logging.info('Model number of weights: %s', model.count_params()) # 로그 정보를 알려줌(출력X)
    base_lr = FLAGS.base_learning_rate * batch_size / 128 #learningRate 선언
    lr_decay_epochs = [(int(start_epoch_str) * FLAGS.train_epochs) // 200 #decay_epochs 선언, (60,120,180)*(200//200)
                       for start_epoch_str in FLAGS.lr_decay_epochs]
    lr_schedule = ub.schedules.WarmUpPiecewiseConstantSchedule( #학습속도 조정(=tf.keras.optimizers.schedules.LearningRateScheduler)
        steps_per_epoch,
        base_lr,
        decay_ratio=FLAGS.lr_decay_ratio,
        decay_epochs=lr_decay_epochs,
        warmup_epochs=FLAGS.lr_warmup_epochs)
    optimizer = tf.keras.optimizers.SGD(lr_schedule, #Gradient descent optimizer 선언
                                        momentum=1.0 - FLAGS.one_minus_momentum,
                                        nesterov=True)
    metrics = { #평가지표 행렬 선언
        'train/negative_log_likelihood': #train 데이터셋의 NLL
            tf.keras.metrics.Mean(),
        'train/accuracy': #train 데이터셋의 accuracy
            tf.keras.metrics.SparseCategoricalAccuracy(),
        'train/loss': #train 데이터셋의 loss
            tf.keras.metrics.Mean(),
        'train/ece': #train 데이터셋의 ece
            rm.metrics.ExpectedCalibrationError(num_bins=FLAGS.num_bins),
        'test/negative_log_likelihood': #test 데이터셋의 NLL
            tf.keras.metrics.Mean(),
        'test/accuracy': #test 데이터셋의 accuracy
            tf.keras.metrics.SparseCategoricalAccuracy(),
        'test/ece': #test 데이터셋의 ece
            rm.metrics.ExpectedCalibrationError(num_bins=FLAGS.num_bins),
    }

    checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer) #checkpoint에 모델과 optimizer 읽을 수 있는 값을 저장
    latest_checkpoint = tf.train.latest_checkpoint(FLAGS.output_dir) # 최근에 저장된 checkpoint 파일의 파일 이름 저장, 없으면 false반환
    initial_epoch = 0 #초기 epoch 선언, 모델을 돌렸을 때 중간에 멈췄다면 이어서 돌릴 수 있음
    if latest_checkpoint: #전에 학습한 체크포인트가 있으면
      checkpoint.restore(latest_checkpoint) #체크포인트를 latest_checkpoint로 update
      logging.info('Loaded checkpoint %s', latest_checkpoint) #로그 정보를 알려줌(출력X)
      initial_epoch = optimizer.iterations.numpy() // steps_per_epoch #initial_epoch을 latest_checkpoint 이후로 설정

### Train
* loss에 대하여 layer의 학습가능한 weight의 gradient 를 저장

In [ ]:
  @tf.function
  def train_step(iterator):
    """Training StepFn."""
    def step_fn(inputs):
      """Per-Replica StepFn."""
      images = inputs['features'] #변수 선언
      labels = inputs['labels'] #라벨 선언

      with tf.GradientTape() as tape: #레이어가 입력에 적용하는 연산은 gradient Tape에 기록, 자동적으로 미분
        logits = model(images, training=True) # logits 저장
        if FLAGS.label_smoothing == 0.:
          negative_log_likelihood = tf.reduce_mean( #sparse_categorical_crossentropy의 모든 텐서 차원의 평균 계산
              tf.keras.losses.sparse_categorical_crossentropy(labels,
                                                              logits,
                                                              from_logits=True))
        
        l2_loss = sum(model.losses) # l2 로스 계산
        loss = negative_log_likelihood + l2_loss # loss 계산
        scaled_loss = loss / strategy.num_replicas_in_sync #loss 스케일링(장치 수로 나누기(=8))

      grads = tape.gradient(scaled_loss, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables)) #optimizer 를 통해 업데이트 되는 gradient를 사용

      probs = tf.nn.softmax(logits) #softmax activation
      metrics['train/ece'].add_batch(probs, label=labels) #train/ece update
      metrics['train/loss'].update_state(loss) #train/loss update
      metrics['train/negative_log_likelihood'].update_state( #train/negative_log_likelihood update
          negative_log_likelihood)
      metrics['train/accuracy'].update_state(labels, logits) #train/accuracy update

    for _ in tf.range(tf.cast(steps_per_epoch, tf.int32)): #올바른 복제본 별 데이터를 단위에 맞춰서 제공
      strategy.run(step_fn, args=(next(iterator),))

### Test

In [ ]:
  @tf.function
  def test_step(iterator, dataset_split, dataset_name, num_steps):
    """Evaluation StepFn."""
    def step_fn(inputs):
      """Per-Replica StepFn."""
      images = inputs['features'] #변수 선언
      labels = inputs['labels'] #라벨 선언
      logits = model(images, training=False) #logits 저장
      probs = tf.nn.softmax(logits) #softmax activation
      negative_log_likelihood = tf.reduce_mean( # NLL 계산
          tf.keras.losses.sparse_categorical_crossentropy(labels, probs))

      if dataset_name == 'clean':
        metrics[f'{dataset_split}/negative_log_likelihood'].update_state( #test/negative_log_likelihood update
            negative_log_likelihood)
        metrics[f'{dataset_split}/accuracy'].update_state(labels, probs) #test/accuracy update
        metrics[f'{dataset_split}/ece'].add_batch(probs, label=labels) #test/ece update

    for _ in tf.range(tf.cast(num_steps, tf.int32)): #올바른 복제본 별 데이터를 단위에 맞춰서 제공
      strategy.run(step_fn, args=(next(iterator),))

### 학습, 검증 진행

In [ ]:
  train_iterator = iter(train_dataset)
  start_time = time.time() #시작 시간 저장
  

  for epoch in range(initial_epoch, FLAGS.train_epochs): #학습, 검증 epoch 진행, (0,200)
    logging.info('Starting to run epoch: %s', epoch)# 로그 정보를 알려줌(출력X)
    train_start_time = time.time()  # 시작 시간 저장
    train_step(train_iterator)  #train-step 함수 시작(학습 진행)
    
    # 시간 계산
    current_step = (epoch + 1) * steps_per_epoch
    max_steps = steps_per_epoch * FLAGS.train_epochs
    time_elapsed = time.time() - start_time
    steps_per_sec = float(current_step) / time_elapsed
    eta_seconds = (max_steps - current_step) / steps_per_sec
    message = ('{:.1%} completion: epoch {:d}/{:d}. {:.1f} steps/s. '
               'ETA: {:.0f} min. Time elapsed: {:.0f} min'.format(
                   current_step / max_steps,
                   epoch + 1,
                   FLAGS.train_epochs,
                   steps_per_sec,
                   eta_seconds / 60,
                   time_elapsed / 60))
    logging.info(message)# 로그 정보를 알려줌(출력X)
    

    datasets_to_evaluate = {'clean': test_datasets['clean']}
    for dataset_name, test_dataset in datasets_to_evaluate.items():
      test_iterator = iter(test_dataset)
      logging.info('Testing on dataset %s', dataset_name)# 로그 정보를 알려줌(출력X)
      logging.info('Starting to run eval at epoch: %s', epoch)# 로그 정보를 알려줌(출력X)
      test_start_time = time.time() #시작 시간 저장
      test_step(test_iterator, 'test', dataset_name, steps_per_eval) #test-step 함수 시작(검증 진행)

      logging.info('Done with testing on %s', dataset_name)# 로그 정보를 알려줌(출력X)


    logging.info('Train Loss: %.4f, Accuracy: %.2f%%',# 로그 정보를 알려줌(출력X)
                 metrics['train/loss'].result(),
                 metrics['train/accuracy'].result() * 100)
    logging.info('Test NLL: %.4f, Accuracy: %.2f%%',# 로그 정보를 알려줌(출력X)
                 metrics['test/negative_log_likelihood'].result(),
                 metrics['test/accuracy'].result() * 100)
    
    total_results = {name: metric.result() for name, metric in metrics.items()} #전체 reslut 저장
    total_results = { # key value 형태로 저장되어 있는 값들 정렬
        k: (list(v.values())[0] if isinstance(v, dict) else v)
        for k, v in total_results.items()
    }
    with summary_writer.as_default(): # summary 로그 기록
      for name, result in total_results.items():
        tf.summary.scalar(name, result, step=epoch + 1)

    for metric in metrics.values(): # metric 초기화
      metric.reset_states()

    if (FLAGS.checkpoint_interval > 0 and #checkpoint_interval 마다 checkpoint 저장, checkpoint_interval=25
        (epoch + 1) % FLAGS.checkpoint_interval == 0):
      checkpoint_name = checkpoint.save(
          os.path.join(FLAGS.output_dir, 'checkpoint'))
      logging.info('Saved checkpoint to %s', checkpoint_name)# 로그 정보를 알려줌(출력X)

### final checkpoint, summary 기록

In [ ]:
  final_checkpoint_name = checkpoint.save( #fianl checkpoint 저장
      os.path.join(FLAGS.output_dir, 'checkpoint'))
  logging.info('Saved last checkpoint to %s', final_checkpoint_name) # 로그 정보를 알려줌(출력X)
  with summary_writer.as_default(): #summary 로그 기록
    hp.hparams({
        'base_learning_rate': FLAGS.base_learning_rate, #base_learning_rate, 0.1
        'one_minus_momentum': FLAGS.one_minus_momentum, #one_minus_momentum, 0.1
        'l2': FLAGS.l2, #l2, 2e-4
    })

# main 함수 실행

In [ ]:
if __name__ == '__main__':
  app.run(main)